This file is a helper file to test the unit tests `testQp2d.cpp` and `testPiecewiseQuadratic.cpp`.

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## Problem Definition
The objective function is:
$$
\begin{cases}
(0.5) (x + 0.5)^2 + (0)xy + (3)(y-0.1)^2, & x < 0\\
(0.8) x^2 - (1.2)xy + y^2, & x \geq 0
\end{cases}
$$
$$
\begin{cases}
(0.5) x^2 + (3)y^2 + (0)xy + (0.5)x - (0.6)y + (0.155), & x < 0\\
(0.8) x^2 + (1)y^2 - (1.2)xy + (0)x + (0)y + (0), & x \geq 0
\end{cases}
$$

The inequality constraints form an octagon with vert/horiz edges length 1 distance 1 from origin, and diagonal edges with length 1/sqrt(2).  Just look at the graph at the very bottom.

## Deriving the analytical solution

For z2,
  $$\begin{align}
    x_2^*(y) &= (+1.2/1.6) y = (0.75) * y \\
    z_2^*(y) &= (0.8) (0.75)^2 y^2 - (1.2) (0.75) y^2 + y^2 \\
              &= 0.55 y^2
    \end{align}$$
For z1,
  $$\begin{align}
    x_1^*(y) &= -0.5 \\
    z_1^*(y) &= 0 + 0 + 3(y-0.1)^2 \\
              &= 3(y-0.1)^2
    \end{align}$$

The first segment starts at $y=-1$.  Pictorally, we can see that the solution rides $z_2$ along $x=0$, so $z_2^*(y) = y^2$ until $z_2^*(y)$ gets better.  That happens at $y=0$, so the first segment is $x=0$ for $y \in [-1, 0]$.

Then we take $z_2^*(y)$ until $x_1^*(y)$ gets better then follows $x_1^*(y)$ until $x_2^*(y)$ gets better again.  These 2 switching points occur at
  $$ \begin{align}
    0.55y^2 &= 3(y-0.1)^2 \\
    (3-0.55)y^2 - 0.6y + 0.03 &= 0 \\
    y &= 0.0700194588625913, 0.17487850032108213
    \end{align}$$

The final segment comes from the inequality constraint $x+y\le 1.5$, so it starts when the line $x=0.75y$ intersects $x+y=1.5$ and continues until $y=1$.  So that's at $y=1.5/1.75=6/7$.

In [ ]:
# Helper code for ^
a, b, c = 3-0.55, -0.6, 0.03
sol1 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
sol2 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
print(sol1, sol2)

# Code

In [ ]:
t = np.linspace(-1, 1, 500)
x, y = np.meshgrid(t, t)
z1 = 0.5 * (x + 0.5)**2 + 3 * (y-0.1)**2
z2 = 0.8 * x**2 - 1.2 * x*y + y**2
z = z1 * (x < 0) + z2 * (x >= 0)
# Check the conversion / canonicalization
vars = [x**2, y**2, x*y, x, y, np.ones_like(x)]
z1a = np.sum([c*v for c, v in zip([0.5, 3, 0, 0.5, -0.6, 0.155], vars)], axis=0)
z2a = np.sum([c*v for c, v in zip([0.8, 1, -1.2, 0, 0, 0], vars)], axis=0)
assert np.allclose(z1, z1a)
assert np.allclose(z2, z2a)

In [ ]:
inequalities = ((x <= 1) & (x >= -1) & (y <= 1) & (y >= -1) &
                (x + y <= 1.5) & (x + y >= -1.5) & (x - y <= 1.5) & (x - y >= -1.5))
z[~inequalities] = np.nan

In [ ]:
# Expected solution, numerical approximation
xmin = t * 0
for i, row in enumerate(z):
    row2 = row.copy()
    row2[np.isnan(row2)] = np.inf
    xmin[i] = t[np.argmin(row2)]

In [ ]:
# Expected solution, analytical solution
y_ = t
xc2, xc3 = 0.0700194588625913, 0.17487850032108213
xmin2 = ((0) * (y_ < 0) +  #
         (0.75 * y_) * (y_ >= 0) * (y_ <= xc2) +  #
         (-0.5) * (y_ > xc2) * (y_ <= xc3) +  #
         (0.75 * y_) * (y_ > xc3) * (y_ <= 6/7) + #
         (1.5 - y_) * (y_ >= 6/7))

In [ ]:
fig = go.Figure(data=go.Contour(x=t, y=t, z=z))
fig.add_trace(go.Scatter(x=xmin, y=t, mode='lines', name='numerical'))
fig.add_trace(go.Scatter(x=xmin2, y=t, mode='lines', name='analytical', line=dict(dash='dash', color='black')))
# Make axes equal by setting aspect ratio to 1
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_layout(width=400, height=300, margin=dict(l=0, r=0, b=0, t=0))
fig.show()